In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import floor

# parametri della simulazione
n_traces = 4
n_ues = 12
time_quantum = 0.01
start_time = 0
end_time = 2.0

# inizializzazione
times = np.arange(start_time, end_time + time_quantum, time_quantum)
mean_throughput = np.zeros((n_ues, len(times)))
power_throughput = np.zeros((n_ues, len(times)))
mean_delay = np.zeros((n_ues, len(times)))
power_delay = np.zeros((n_ues, len(times)))
curr_run = 0
in_file = "0"
prev_file = "1"

In [ ]:
if (curr_run == n_traces):
    print("Tutte le tracce sono state analizzate!")
elif (prev_file is in_file):
    print("File già analizzato!")
else:
    print "Run: " + str(curr_run) + " \nBisogna eseguire ancora " + str(n_traces - curr_run - 1) + " run."
    curr_run += 1

    # file di input
    in_path = "/home/luca/Scrivania/TLCnet_prj/ns3-mmwave/output8/"
    in_file = "mcDlPdcpStats_5_13_23_12_2017_06_46_29.txt"

    delay = np.zeros((n_ues, len(times)))

    ues_rx = [[] for _ in range(n_ues)]

    with open(in_path + in_file, 'r') as f: 
        prev_line = f.readline()
        for line in f:
            # elimino duplicati consecutivi
            if (line != prev_line):
                prev_line = line
                tokens = line.rstrip().split(' ')
                if tokens[0] == "Rx":
                    lcid = int(tokens[5])
                    if lcid > 1:
                        time = float(tokens[1])
                        if (time < end_time + time_quantum):
                            ue = int(tokens[3]) - 1
                            lcid = int(tokens[5])
                            pck_size = float(tokens[6]) / (10e6 * time_quantum)
                            pck_delay = float(tokens[7]) / 10e6
                            ues_rx[ue].append((time, pck_size, pck_delay))
        f.close()   

    for i in range(n_ues):
        curr_throughput = np.zeros(len(times))
        curr_delay = np.zeros(len(times))
        count = np.zeros(len(times))
        for pck in ues_rx[i]:
            mean_throughput[i, int(floor(pck[0] / time_quantum))] += pck[1] / n_traces
            curr_throughput[int(floor(pck[0] / time_quantum))] += pck[1]
            curr_delay[int(floor(pck[0] / time_quantum))] += pck[2]
            count[int(floor(pck[0] / time_quantum))] += 1
        count = count.clip(min = 1)
        power_throughput[i] += np.divide(np.square(curr_throughput), n_traces)
        mean_delay[i] += np.divide(np.divide(curr_delay, count), n_traces)
        power_delay[i] += np.divide(np.square(np.divide(curr_delay, count)), n_traces)
        
    prev_file = '%s' % in_file
    print("Finito!")

In [ ]:
if (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run."
else:
    # calcolo lo scarto quadratico medio con la formula sqrt(E(x^2) - E(x)^2)
    sigma_throughput = np.sqrt(power_throughput - np.square(mean_throughput))
    sigma_delay = np.sqrt(power_delay - np.square(mean_delay))
    
    # calcolo delta per l'intervallo di confidenza al 95%
    delta_throughput = 1.96 * np.divide(sigma_throughput, np.sqrt(n_traces) * np.ones(len(times)))
    delta_delay = 1.96 * np.divide(sigma_delay, np.sqrt(n_traces) * np.ones(len(times)))

    # calcolo la media temporale
    avg_throughput = np.mean(mean_throughput, axis = 1)
    avg_delay = np.mean(mean_delay, axis = 1)
    print("Finito!")

In [ ]:
if (curr_run != n_traces):
    print "Bisogna eseguire ancora " + str(n_traces - curr_run) + " run."
else:
    # plot CI
    for i in range(n_ues):
        fig = plt.figure(figsize = (20, 10))
        ax1 = plt.gca()
        ax1.set_xlim(start_time, end_time + time_quantum)
        ax1.xaxis.set_ticks(times)
        ax1.xaxis.set_ticklabels(times)

        plt.subplot(2, 1, 1)
        plt.title("Throughput ue" + str(i + 1))
        plt.xlabel("Time [s]")
        plt.ylabel('Througput [Mb/s]')
        plt.grid(True)

        plt.plot(times, mean_throughput[i], lw = 2, color = 'b')
        plt.plot(times, mean_throughput[i] + delta_throughput[i], '--', lw = 2, color = 'b')
        plt.plot(times, avg_throughput[i] * np.ones(len(times)), '--', lw = 2, color = 'g')
        plt.plot(times, mean_throughput[i] - delta_throughput[i], '--', lw = 2, color = 'b')

        plt.legend(['throughput', 'CI 95%', "mean"])

        plt.subplot(2, 1, 2)
        plt.title("Delay ue" + str(i + 1))
        plt.xlabel("Time [s]")
        plt.ylabel('Delay [ms]')
        plt.grid(True)

        plt.plot(times, mean_delay[i], lw = 2, color = '#800080')
        plt.plot(times, mean_delay[i] + delta_delay[i], '--', lw = 2, color = '#800080')
        plt.plot(times, avg_delay[i] * np.ones(len(times)), '--', lw = 2, color = 'r')
        plt.plot(times, mean_delay[i] - delta_delay[i], '--', lw = 2, color = '#800080')

        plt.legend(['delay', 'CI 95%', "mean"])

        plt.tight_layout()
        plt.show()